# Load API Key and OPENAI Package

Loads all of the packages that will be used and the OpenAI API Key that will be used. Store your OpenAI API Key in a .env file.
If you are missing any of these packages run the following commands
```
$   pip3 install PyPDF2
$   pip3 install python.dotenv
$   pip3 install openai
$   pip3 install nltk
```

In [13]:
import os
import openai
import PyPDF2
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
openai.organization = os.getenv("ORGANIZATION_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [6]:
# Sample API Call
# response = openai.Completion.create(
#     engine="text-davinci-002",  # Specify the engine (model) you want to use
#     prompt="Translate the following English text to French: 'Hello, how are you?'",
#     max_tokens=50,  # Limit the length of the generated text
# )

# Lesson Proposals

Lesson Proposals will be extracted using the PyPDF2 Package. You must have the lessons downloaded locally for this code to work and place it inside of a 
Lesson_Proposal folder. It will extract all of the text and put it into the <i style="color: red"><b>proposal_text</b></i> variable.

## Retrieving Lesson Proposals

In [26]:
# Stores the path of All the PDFs in the Lesson_Proposals Folder
pdf_file_paths = []

proposal_list = os.listdir("./Lesson_Proposals/")

# Extracts all the PDF Paths
for i in proposal_list:
    if i == ".DS_Store":
        continue
    pdf_file_paths.append(f"Lesson_Proposals/{i}")

.DS_Store


In [27]:
print(pdf_file_paths)

['Lesson_Proposals/114233851983_A_Path_to_Open_Inclusive_and_Collaborative_Science_for_Librarians.pdf', 'Lesson_Proposals/114233826722_Open_Hardware_for_librarians.pdf', 'Lesson_Proposals/114219657654_Data_Management_and_Sharing_Plans_for_Librarians_101.pdf', 'Lesson_Proposals/114229483598_Research Community Outreach with Open Science Team Agreements_Open_Science_Team_Agreements-Lessons_for_Librarians_in_Open_Science_Proposal.pdf', 'Lesson_Proposals/114232854610_Understanding_CARE_Principles_for_research_data.pdf', 'Lesson_Proposals/114233727582_Reproducible_research_workflows_2023_01_31.pdf', 'Lesson_Proposals/114205095243_Open_Qualitative_Research.pdf']


In [28]:
# EXTRACT TEXT FROM EACH PROPOSAL

lesson_proposal = []

for i in pdf_file_paths:
    pdf_file = open(i, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    text = ''
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()
    
    pdf_file.close()
    lesson_proposal.append(text)


## Cleaning Proposal Data

In [29]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lawrencelee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lawrencelee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
from nltk import sent_tokenize, word_tokenize
proposal_sentence = sent_tokenize(lesson_proposal[0])
proposal_tokens = word_tokenize(lesson_proposal[0])

In [34]:
print(proposal_sentence)
print(proposal_tokens)

['A Path to Open, Inclusive, and\nCollaborative Science for Librarians\nKeywords\nOpen access, open science, open data, open educational resources, Spanish-speaking\ncommunities\nLesson Audience\nLibrary and information science professionals who are interested in encouraging the use\nof open research practices, easing access to open research resources, and generating\nmore inclusive and accessible research networks for non-native English speakers.', 'Practitioners in the fields of information science who want to learn about scientific data\nmanagement and stewardship from a Latin American context.', 'Description\nThis 3-hour lesson is organized into three main sections covering: open, collaborative,\nand inclusive science;\nFAIR\nand\nCARE\nprinciples; and\nLatin American initiatives and\npractices.', 'The declaration of 2023 as the\nYear of Open Science\nby NASA  and other federal US\nagencies reflects the belief that open science is a pillar to ensure information access\nand the demo

In [56]:
import string
table = str.maketrans('', '', string.punctuation)
tokens = [w.translate(table) for w in proposal_tokens]

In [57]:
print(tokens)

['A', 'Path', 'to', 'Open', '', 'Inclusive', '', 'and', 'Collaborative', 'Science', 'for', 'Librarians', 'Keywords', 'Open', 'access', '', 'open', 'science', '', 'open', 'data', '', 'open', 'educational', 'resources', '', 'Spanishspeaking', 'communities', 'Lesson', 'Audience', 'Library', 'and', 'information', 'science', 'professionals', 'who', 'are', 'interested', 'in', 'encouraging', 'the', 'use', 'of', 'open', 'research', 'practices', '', 'easing', 'access', 'to', 'open', 'research', 'resources', '', 'and', 'generating', 'more', 'inclusive', 'and', 'accessible', 'research', 'networks', 'for', 'nonnative', 'English', 'speakers', '', 'Practitioners', 'in', 'the', 'fields', 'of', 'information', 'science', 'who', 'want', 'to', 'learn', 'about', 'scientific', 'data', 'management', 'and', 'stewardship', 'from', 'a', 'Latin', 'American', 'context', '', 'Description', 'This', '3hour', 'lesson', 'is', 'organized', 'into', 'three', 'main', 'sections', 'covering', '', 'open', '', 'collaborative

In [59]:
tokens = [word.lower() for word in tokens]
# print(tokens)

In [53]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
tokens = [w for w in tokens if not w in stop_words]

# Prompting GPT


OpenAI's API currently doesn't support a continuous chat as far as im concerned. There are a lot of repositories that address this problem but I think the solution I will be going with is a long continuous chain, starting with providing a rubric to score similiarities, providing an output format, then ultimately providing all of the proposals. We can test this with the online version GPT prior to using the API to see if it generates the result we want.

## Creating the Prompt

### Opening a Chat Log

In [ ]:
# class Chat:

### Providing a Rubric

You can find the Rubric that was used to score similarity [here](https://docs.google.com/document/d/1x18mVubT2H4Gj_GvDM3nUupCqYQHaH8Qk8UleAQjPyU/edit).

In [ ]:
# Telling GPT to use this Rubric when Grading the Proposals

rubric_message = """

"""

### Telling GPT How to Output the Results

In [ ]:
format_message = """
Please respond in the following format: 

1. Proposal 1 Title
    Most Similar: (Proposal Most Similar to)
        3-5 Sentences of Context comparing the 2 proposals
        Rubric Score:
            ...
    Least Similar: (Proposal Least Similar to)
        3-5 Sentences of Content comparing the 2 proposals
        Rubric Score:
            ...
2. Proposal 2 Title
    Same format as above
3. The same format until all proposals have been considered
"""


### Combining all of the messages

We will take all of the different messages created above and combine them together into one singular prompt that we can send to GPT.
It will HOPEFULLY output an appropriate response.

In [ ]:
prompt = f"""
You are going to receive several lesson proposals and your job is to compare them to find proposals that are most similar to each other. 
You will have a total of 7 proposals, A, B, C, etc. You will compare the proposals and see which is the most similar and least similar to each individual 
proposal. Each proposal will go through an individual comparison against each other. You will use the rubric that will be provided as a way to score
the similarities between each proposal. This score will be used in your response.

The Rubric:
{rubric_message}

{format_message}

Here are the following proposals

Proposal 1:
{lesson_proposal[0]}

Proposal 2:
{lesson_proposal[1]}

Proposal 3:
{lesson_proposal[2]}

Proposal 4:
{lesson_proposal[3]}

Proposal 5:
{lesson_proposal[4]}

Proposal 6:
{lesson_proposal[5]}

Proposal 7:
{lesson_proposal[6]}

"""

## Sending the Prompt

### Sending API Call

In [ ]:
# Using OPENAI's API to prompt gpt-4 to analyze the text
response = openai.Completion.create(
    engine="gpt-4",  # Specify the engine (model) you want to use
    prompt="Translate the following English text to French: 'Hello, how are you?'",
    max_tokens=5000,  # Limit the length of the generated text
)

### Extracting the Message

The following code will create a new file/overwrite the "GPT_Analysis.txt" with GPT's text analysis of the lesson proposals.

In [ ]:
# print(response['choices'][0]['text'])
GPT_Analysis = open("GPT_Analysis.txt", 'w')
GPT_Analysis.write(response['choices'][0]['text'])
GPT_Analysis.close()